In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

## ODE Solution

In [ ]:
def forward_euler(ddt, u0, T, *args):
    u = np.empty((T.size, u0.size))
    u[0] = u0
    for i in range(1, T.size):
        u[i] = u[i-1] + (T[i] - T[i-1]) * ddt(u[i-1], T[i-1], *args)
    return u

def ddt(u, t, params):
    beta, rho, sigma = params
    x, y, z = u
    return np.array([sigma*(y-x), x*(rho-z)-y, x*y-beta*z])

def solve_ode(N, dt, u0, params=[8/3, 28, 10]):
    """
        Solves the ODEs for N time steps starting from u0.
        Returned values are normalized.

        Args:
            N: number of time steps
            u0: initial condition
            norm: normalisation factor of u0 (None if not normalised)
            params: parameters for ODE
        Returns:
            normalized time series of shape (N+1, u0.size)
    """

    T = np.arange(N+1) * dt
    U = forward_euler(ddt, u0, T, params)

    return U

## ESN 

In [ ]:
## ESN with bias architecture

# @njit(parallel=False)
def step(x_pre, u, sigma_in, rho):
    """ Advances one ESN time step.
        Args:
            x_pre: reservoir state
            u: input
        Returns:
            new augmented state (new state with bias_out appended)
    """
    # input is normalized and input bias added
    u_augmented = np.hstack((u/norm, np.array([bias_in]))) 
    # hyperparameters are explicit here
    x_post      = np.tanh(np.dot(u_augmented*sigma_in, Win) + rho*np.dot(x_pre, W)) 
    # output bias added
    x_augmented = np.hstack((x_post, np.array([bias_out])))

    return x_augmented

# @njit(parallel=False)
def open_loop(U, x0, sigma_in, rho):
    """ Advances ESN in open-loop.
        Args:
            U: input time series
            x0: initial reservoir state
        Returns:
            time series of augmented reservoir states
    """
    N  = U.shape[0]
    Xa = np.empty((N+1, N_units+1))
    Xa[0] = np.hstack((x0, np.array([bias_out])))
    for i in np.arange(1,N+1):
        Xa[i] = step(Xa[i-1,:N_units], U[i-1], sigma_in, rho)

    return Xa

# @njit(parallel=False)
def closed_loop(N, x0, Wout, sigma_in, rho):
    """ Advances ESN in closed-loop.
        Args:
            N: number of time steps
            x0: initial reservoir state
            Wout: output matrix
        Returns:
            time series of prediction
            final augmented reservoir state
    """
    xa = x0.copy()
    Yh = np.empty((N+1, dim))
    Yh[0] = np.dot(xa, Wout)
    for i in np.arange(1,N+1):
        xa = step(xa[:N_units], Yh[i-1], sigma_in, rho)
        Yh[i] = np.dot(xa, Wout)

    return Yh, xa

# @njit(parallel=False)
def train(U_washout, U_train, Y_train, tikh, sigma_in, rho):
    """ Trains ESN.
        Args:
            U_washout: washout input time series
            U_train: training input time series
            tikh: Tikhonov factor
        Returns:
            time series of augmented reservoir states
            optimal output matrix
    """
    
    LHS = 0
    RHS = 0
    
    N  = U_train[0].shape[0]    
    Xa  = np.zeros((U_washout.shape[0], N+1, N_units+1))
    
    for i in range(U_washout.shape[0]):
        
        ## washout phase
        xf_washout = open_loop(U_washout[i], np.zeros(N_units), sigma_in, rho)[-1,:N_units]

        ## open-loop train phase
        Xa[i] = open_loop(U_train[i], xf_washout, sigma_in, rho)
    
        ## Ridge Regression
        LHS  += np.dot(Xa[i,1:].T, Xa[i,1:])
        RHS  += np.dot(Xa[i,1:].T, Y_train[i])
    
    Wout = np.zeros((tikh.size, N_units+1,dim))
    for j in np.arange(tikh.size):
        Wout[j] = np.linalg.solve(LHS + tikh[j]*np.eye(N_units+1), RHS)

    return Xa[0], Wout, LHS, RHS

In [ ]:
def predictability_horizon(xa, Y, Wout):
    """ Compute predictability horizon. It evolves the network until the
        error is greater than the threshold. Before that it initialises
        the network by running a washout phase.
        
        Args:
            threshold: error threshold
            U_washout: time series for washout
            Y: time series to compare prediction
        
        Returns:
            predictability horizon (in time units, not Lyapunov times)
            time series of normalised error
            time series of prediction
    """
    
    # calculate denominator of the normalised error
    error_denominator = np.mean(np.sum((Y)**2, axis=1))

    N     = Y.shape[0]
    E     = np.zeros(N)
    Yh    = np.zeros((N, dim))
    Yh[0] = np.dot(xa, Wout)

    for i in range(1, N):
        # advance one step
        xa = step(xa[:N_units], Yh[i-1])
        Yh[i] = np.dot(xa, Wout)

        # calculate error
        error_numerator = np.sum(((Yh[i]-Y[i]))**2)
        E[i] = np.sqrt(error_numerator/error_denominator)

        if E[i] > threshold:
            break
            
    return i/N_lyap